In [ ]:
# Importar as bibliotecas

import json
import os
import pandas

In [ ]:
# Trazer as opções de configuração do JSON

with open('../options.json', 'r') as file:
    data = json.load(file)
file.close()

In [ ]:
# Lista os investimentos escolhidos

choosed_investments = data["INVESTMENTS"]
choosed_investments = list(choosed_investments.keys())

In [ ]:
# Recolhe os investimentos

data_dir = os.path.abspath('../archives/archives-csv/')
frames = []

# Ajuste do arquivos

for x in range (1, 3):
    global list_my_investments
    
    archive = 'teste' + str(x) + '.csv'
    list_investments_month = pandas.read_csv(os.path.join(data_dir, archive), sep=';', encoding='utf-8-sig')
    list_my_investments_month = list_investments_month.query('CNPJ_FUNDO_CLASSE in @choosed_investments').copy()

    frames.append(list_my_investments_month)

if frames:
    list_my_investments = pandas.concat(frames, ignore_index=True)

list_my_investments = list_my_investments.sort_values(by=["CNPJ_FUNDO_CLASSE", "DT_COMPTC"])

In [ ]:
# Reseta o índice da coluna de identificação

list_my_investments = list_my_investments.reset_index()

In [ ]:
# Criação da coluna do nome dos fundos

started_value_cota = 0
preview_cnpj = ''

def search_name(element):
    return data['INVESTMENTS'][element]['Fundo']

list_my_investments['NOME_FUNDOS'] = list_my_investments.CNPJ_FUNDO_CLASSE.map(search_name)

In [ ]:
# Criação da coluna da valorização de cotas

started_value_cota = 0
preview_cnpj = ''

def string_to_number(element):
    return int(element)

def is_first_date_previous_than_second (investment_date_start, investment_date_row):
    start_year, start_month, start_day = map(string_to_number, str(investment_date_start).split('-'))
    row_year, row_month, row_day = map(string_to_number, str(investment_date_row).split('-'))

    if start_year > row_year:
        return False
    elif start_year < row_year:
        return True

    if start_month > row_month:
        return False
    elif start_month < row_month:
        return True
    
    if start_day > row_day:
        return False
    elif start_day < row_day:
        return True
   
    return False

def calculate_valuation(row):
    global started_value_cota
    global preview_cnpj

    # Tratar o caso de loop de CNPJ
    current_cnpj = row['CNPJ_FUNDO_CLASSE']
    if preview_cnpj != current_cnpj:
        started_value_cota = 0
    preview_cnpj = current_cnpj

    investment_date_start = data['INVESTMENTS'][row['CNPJ_FUNDO_CLASSE']]['HistoricoCompraDatas'][0]
    investment_date_row = row['DT_COMPTC'][2:]

    boolean_test = is_first_date_previous_than_second(investment_date_start, investment_date_row)

    if boolean_test == False:

        # Isso pode dar erro, estou seguindo a regra que um usuário só pode comprar quando cotado o valor, pois se não, como afirmar qual foi a data?
        if investment_date_start == investment_date_row:
            started_value_cota = float(row['VL_QUOTA'])
        return 0.00
    
    valuation_percentual = ((float(row['VL_QUOTA']) / started_value_cota) - 1) * 100
    
    return round(valuation_percentual, 2)

list_my_investments['VALUATION_PERCENT'] = list_my_investments.apply(calculate_valuation, axis=1)

list_my_investments = list_my_investments[["CNPJ_FUNDO_CLASSE", "NOME_FUNDOS", "DT_COMPTC", "VL_QUOTA", "VALUATION_PERCENT", "VL_TOTAL","VL_PATRIM_LIQ", "CAPTC_DIA", "RESG_DIA","NR_COTST"]].copy()

display(list_my_investments)

In [ ]:
# Armanzenar os dados formatados da tabela

data_dir = os.path.join(os.path.abspath('../archives/'), 'data')
print(data_dir)

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

list_my_investments.to_csv(os.path.join(data_dir, 'data.csv'))